In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
from loguru import logger
from matbench.bench import MatbenchBenchmark
from matbench.constants import CLF_KEY

from gptchem.gpt_regressor import GPTRegressor
from gptchem.tuner import Tuner

logger.enable("gptchem")

In [3]:
import time

import decorator


def retry(howmany, *exception_types, **kwargs):
    timeout = kwargs.get("timeout", 0.0)  # seconds

    @decorator.decorator
    def tryIt(func, *fargs, **fkwargs):
        for _ in range(howmany):
            try:
                return func(*fargs, **fkwargs)
            except exception_types or Exception as e:
                print(e)
                if timeout is not None:
                    time.sleep(timeout)

    return tryIt

In [4]:
mb = MatbenchBenchmark(
    autoload=True,
    subset=[
          "matbench_expt_gap",
        #"matbench_steels",
    ],
)

2023-01-31 00:22:47 INFO     Initialized benchmark 'matbench_v0.1' with 1 tasks: 
['matbench_expt_gap']


In [5]:
@retry(3, timeout=5)
def train_test_fold(task, fold):
    regressor = GPTRegressor(
        task.metadata["target"], Tuner(n_epochs=8, learning_rate_multiplier=0.02, wandb_sync=False)
    )
    train_inputs, train_outputs = task.get_train_and_val_data(fold)

    # train and validate your model
    regressor.fit(train_inputs, train_outputs.values)

    # Get testing data
    test_inputs = task.get_test_data(fold, include_target=False)

    # Predict on the testing data
    # Your output should be a pandas series, numpy array, or python iterable
    # where the array elements are floats or bools
    predictions = regressor.predict(test_inputs)

    # Record your data!
    task.record(fold, predictions)
    return predictions

In [6]:
from collections import defaultdict

In [7]:
predictions = defaultdict(list)

for task in mb.tasks:
    task.load()

    for fold_ind, fold in enumerate(task.folds):
        if task.is_recorded[fold_ind]:
            print(f"Skipping fold {fold_ind} of {task.dataset_name}")
            continue
        pred = train_test_fold(task, fold)
        predictions[task.dataset_name].append(pred)
        train_inputs, train_outputs = task.get_train_and_val_data(fold)

    # print(f"{task.dataset_name}: MAE  {task.scores['mae']['mean']}")

2023-01-31 00:22:47 INFO     Dataset matbench_expt_gap already loaded; not reloading dataset.


Upload progress: 100%|██████████| 421k/421k [00:00<00:00, 682Mit/s]


Uploaded file from /Users/kevinmaikjablonka/git/kjappelbaum/gptchem/experiments/04_regression/matbench/out/20230131_002247/train.jsonl: file-fAftUVO7Wk77PeisCF4S2u2L


2023-01-31 00:22:51.753 | DEBUG    | gptchem.tuner:tune:188 - Requested fine tuning. {
  "created_at": 1675120971,
  "events": [
    {
      "created_at": 1675120971,
      "level": "info",
      "message": "Created fine-tune: ft-fHeO00YciXevc2ACSAHXOT57",
      "object": "fine-tune-event"
    }
  ],
  "fine_tuned_model": null,
  "hyperparams": {
    "batch_size": null,
    "learning_rate_multiplier": 0.02,
    "n_epochs": 8,
    "prompt_loss_weight": 0.01
  },
  "id": "ft-fHeO00YciXevc2ACSAHXOT57",
  "model": "ada",
  "object": "fine-tune",
  "organization_id": "org-TFRJXw3PPQocOWbu71eI2t9U",
  "result_files": [],
  "status": "pending",
  "training_files": [
    {
      "bytes": 420861,
      "created_at": 1675120971,
      "filename": "/Users/kevinmaikjablonka/git/kjappelbaum/gptchem/experiments/04_regression/matbench/out/20230131_002247/train.jsonl",
      "id": "file-fAftUVO7Wk77PeisCF4S2u2L",
      "object": "file",
      "purpose": "fine-tune",
      "status": "uploaded",
      "

In [ ]:
task.scores

{'mae': {'mean': 142.05139272913465,
  'max': 164.16984126984127,
  'min': 116.28548387096774,
  'std': 15.649240031586015},
 'rmse': {'mean': 203.6908550641379,
  'max': 228.16760652350862,
  'min': 178.0547437565019,
  'std': 17.405743765520075},
 'mape': {'mean': 0.10162651241009144,
  'max': 0.11943686321489388,
  'min': 0.08450077530845235,
  'std': 0.011543892832947062},
 'max_error': {'mean': 678.9399999999999,
  'max': 765.0999999999999,
  'min': 565.5999999999999,
  'std': 64.39293750094028},
 '_ipython_canary_method_should_not_exist_': {}}

In [ ]:
mb.to_file("gpt_expt_gap_bench.json.gz")

2023-01-30 18:53:04 INFO     Successfully wrote MatbenchBenchmark to file 'gpt_steel_bench.json.gz'.
